# Example stitching workflow

Using a small example dataset with two positions.

In [ ]:
VISUALIZE_USING_NAPARI = False

In [ ]:
import os, tempfile
import numpy as np

from multiview_stitcher import sample_data, msi_utils

import dask.diagnostics

In [ ]:
if VISUALIZE_USING_NAPARI:
    from napari_stitcher import viewer_utils
    import napari
    viewer = napari.Viewer()

In [ ]:
# Load dataset from file

from multiview_stitcher import io

filename = '../test-datasets/mosaic_test.czi'
sims = io.read_mosaic_image_into_list_of_spatial_xarrays(filename)
msims = [msi_utils.get_msim_from_sim(sim, scale_factors=[]) for sim in sims]
print(f"Loaded {len(msims)} positions from {filename}. Transform key is {io.METADATA_TRANSFORM_KEY}.")

# Test with generated dataset

# sims = sample_data.generate_tiled_dataset(
#     ndim=2,
#     N_t=1,
#     N_c=1,
#     tile_size=1000,
#     tiles_x=5,
#     tiles_y=5,
#     zoom=4,
#     overlap=30,
#     drift_scale=0,
#     shift_scale=15,
# )

In [ ]:
if VISUALIZE_USING_NAPARI:
    
    layer_tuples = viewer_utils.create_image_layer_tuples_from_msims(
        msims, n_colors=3, transform_key='affine_metadata')
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, layer_tuples)
    viewer.reset_view()

In [ ]:
from multiview_stitcher import registration, fusion

import importlib
registration = importlib.reload(registration)

print('Registering...')
with dask.diagnostics.ProgressBar():
    params = registration.register(
        msims, transform_key='affine_metadata', reg_channel_index=0)
    
for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

print('Fusing...')
with dask.diagnostics.ProgressBar():
    fused = fusion.fuse(
        [msi_utils.get_sim_from_msim(msim) for msim in msims],
        transform_key='affine_registered')

mfused = msi_utils.get_msim_from_sim(fused, scale_factors=[])

In [ ]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=2)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='pre-registered view')
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused, transform_key='affine_registered',
        name_prefix='registered view')
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

In [ ]:
with dask.diagnostics.ProgressBar():
    io.save_sim_as_tif('fused.tif', fused)